# Generating Counterfactuals

In this notebook, we will focus on generating counterfactuals from individual
datapoints. This will be implemented for the following models:

- Naive Bayes
- Fair Bayesian Network
- Fair Random Forest Classifier

By generating counterfactuals, we hope to gain insight into how the model uses
the different attributes in it's decisions.

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from forseti.bayesnet import latentLabelClassifier, interpretableNaiveBayes
import pandas as pd

df = pd.read_csv('data/adult.csv')
    
clf = interpretableNaiveBayes()

tmp = df
label = 'income'

clf.train(
    label,
    tmp,
    'NB'
)

In [3]:
X = clf.generateCounterfactuals()
X

14015    None
14015    None
14015    None
14015    None
14015    None
         ... 
14015    None
14015    None
14015    None
14015    None
14015    None
Length: 100, dtype: object